In [2]:
!pip install torch torchvision pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 46.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 60.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 36.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 28.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import f1_score
from tqdm import tqdm

print("Current working directory:", os.getcwd())

# 1. Load the CSV file
csv_path = 'train.csv'
print(f"Attempting to load CSV from: {os.path.abspath(csv_path)}")
df = pd.read_csv(csv_path)
print(f"Successfully loaded CSV with {len(df)} entries")

# 2. Custom Dataset Class
class ImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        try:
            img_name = os.path.join(self.image_dir, self.df.iloc[idx, 0] + '.jpg')
            image = Image.open(img_name).convert('RGB')
            label = self.df.iloc[idx, 1]
            if self.transform:
                image = self.transform(image)
            return image, label
        except FileNotFoundError:
            print(f"Error: File not found: {img_name}")
            raise
        except Exception as e:
            print(f"Error loading image {img_name}: {str(e)}")
            raise

# 3. Enhanced Data Transformations with Augmentation
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Create Dataset and DataLoader
image_dir = 'Images'
print(f"Image directory: {os.path.abspath(image_dir)}")
full_dataset = ImageDataset(df, image_dir, transform=None)  # We'll apply transforms later
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Apply transforms
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

print(f"Dataset split - Train: {train_size}, Validation: {val_size}, Test: {test_size}")

# 5. Model (Vision Transformer)
def create_model(num_classes):
    model = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
    
    # Replace the last layer of the classifier
    num_ftrs = model.heads[-1].in_features
    model.heads[-1] = nn.Linear(num_ftrs, num_classes)
    
    return model

# 6. Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device):
    best_val_f1 = 0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")
        print(f"Val F1-Score: {val_f1:.4f}")
        
        scheduler.step(val_loss)
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), 'best_model_vit.pth')

    return model

if __name__ == '__main__':
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # Verify data loading
    print("Verifying data loading...")
    for images, labels in train_loader:
        print("Batch shape:", images.shape)
        print("Labels:", labels)
        break  # Just check the first batch

    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create model
    num_classes = 42  # Assuming 42 classes as in the original code
    model = create_model(num_classes)
    model.to(device)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.05)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)

    # Train model
    num_epochs = 100
    print(f"Starting training for {num_epochs} epochs...")
    model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device)

    # 8. Evaluation
    print("Loading best model for evaluation...")
    model.load_state_dict(torch.load('best_model_vit.pth'))
    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    # 9. Calculate F1-Score
    test_f1 = f1_score(test_labels, test_preds, average='weighted')
    print('Test F1-Score:', test_f1)

Current working directory: /workspace/hackathon-online-agriculture-classification
Attempting to load CSV from: /workspace/hackathon-online-agriculture-classification/train.csv
Successfully loaded CSV with 38511 entries
Image directory: /workspace/hackathon-online-agriculture-classification/Images
Dataset split - Train: 30808, Validation: 3851, Test: 3852
Verifying data loading...
Batch shape: torch.Size([32, 3, 224, 224])
Labels: tensor([19, 38, 32, 26, 39, 18, 27, 37, 19, 19, 19, 41, 39, 33, 29,  2, 39, 19,
         0, 14,  2, 32, 41, 39, 41, 29,  3, 39, 23, 26, 41, 22])
Using device: cuda
Starting training for 100 epochs...


Epoch 1/100 - Validation: 100%|██████████| 121/121 [00:06<00:00, 17.70it/s]


Epoch 1/100
Train Loss: 0.2050
Val Loss: 0.0542
Val F1-Score: 0.9810


Epoch 2/100 - Validation: 100%|██████████| 121/121 [00:06<00:00, 17.63it/s]


Epoch 2/100
Train Loss: 0.0538
Val Loss: 0.0799
Val F1-Score: 0.9777


...

Epoch 23/100 - Validation: 100%|██████████| 121/121 [00:06<00:00, 17.45it/s]


Epoch 23/100
Train Loss: 0.0000
Val Loss: 0.0134
Val F1-Score: 0.9961


Epoch 24/100 - Validation: 100%|██████████| 121/121 [00:06<00:00, 17.37it/s]


Epoch 24/100
Train Loss: 0.0000
Val Loss: 0.0135
Val F1-Score: 0.9961


Epoch 25/100 - Training:  91%|█████████▏| 879/963 [02:11<00:12,  6.82it/s]

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import vit_b_16
from PIL import Image
import os
import csv
import pandas as pd
from tqdm import tqdm
from torchvision.models import swin_t, Swin_T_Weights

# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create the model (same as in the training script)
def create_model(num_classes):
    model = vit_b_16()
    
    # Replace the last layer of the classifier
    num_ftrs = model.heads[-1].in_features
    model.heads[-1] = nn.Linear(num_ftrs, num_classes)
    
    return model

# Create the model
num_classes = 42  # Make sure this matches the number of classes in your trained model
model = create_model(num_classes)

# Load the state dictionary
state_dict = torch.load('best_model_vit_h14.pth', map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

# Set up the image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to predict the class of a single image
def predict_image(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()

# Read the test.csv file
test_csv_path = 'test.csv'
df = pd.read_csv(test_csv_path)

# Ensure the first column contains image names
image_column = df.columns[0]
image_names = df[image_column].tolist()

# Set the path to the test folder containing the images
test_folder = 'Images'  # Replace with your test folder path if different

# Predict classes for all images and store results
results = []
for image_name in tqdm(image_names, desc="Predicting"):
    # Add .jpg extension to the image name
    image_file = image_name + '.jpg'
    image_path = os.path.join(test_folder, image_file)
    
    # Check if the image file exists
    if os.path.exists(image_path):
        predicted_class = predict_image(image_path, model, transform)
        results.append((image_name, predicted_class))
    else:
        print(f"Warning: Image {image_file} not found in {test_folder}")
        results.append((image_name, "Not Found"))

# Write results to CSV file
output_csv = 'predictions2.csv'
with open(output_csv, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image Name', 'Predicted Class'])  # Write header
    csv_writer.writerows(results)

print(f"Predictions completed. Results saved to {output_csv}")

RuntimeError: Error(s) in loading state_dict for SwinTransformer:
	Missing key(s) in state_dict: "features.0.0.weight", "features.0.0.bias", "features.0.2.weight", "features.0.2.bias", "features.1.0.norm1.weight", "features.1.0.norm1.bias", "features.1.0.attn.relative_position_bias_table", "features.1.0.attn.relative_position_index", "features.1.0.attn.qkv.weight", "features.1.0.attn.qkv.bias", "features.1.0.attn.proj.weight", "features.1.0.attn.proj.bias", "features.1.0.norm2.weight", "features.1.0.norm2.bias", "features.1.0.mlp.0.weight", "features.1.0.mlp.0.bias", "features.1.0.mlp.3.weight", "features.1.0.mlp.3.bias", "features.1.1.norm1.weight", "features.1.1.norm1.bias", "features.1.1.attn.relative_position_bias_table", "features.1.1.attn.relative_position_index", "features.1.1.attn.qkv.weight", "features.1.1.attn.qkv.bias", "features.1.1.attn.proj.weight", "features.1.1.attn.proj.bias", "features.1.1.norm2.weight", "features.1.1.norm2.bias", "features.1.1.mlp.0.weight", "feature